In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn/;


kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn

In [44]:
sub route_find {
    my ($map, $start, $end, $steps) = @_;
    my %connections = %$map;

    my @paths = { path => [ $start ], p => $start };
    foreach (1 .. $steps) {
        @paths = map {
            my $path = $_;
                map { { path => [ @{$path->{path}}, $_ ], p => $_ } }
                @{$connections{$path->{p}}}
        }
        @paths;
    }

    return
        map $_->{path},
        grep $_->{p} eq $end,
        @paths;
}

sub process_day23_part1 {
    my ($input) = @_;
    my @pairs = grep $_, split "\n", $input;
    my %connections;
    foreach (@pairs) {
        my ($from, $to) = split '-';
        push @{$connections{$from}}, $to;
        push @{$connections{$to}}, $from;
    }

    my @tri_loops =
        uniq
        map { join ',', sort @{$_}[1 .. 3] }
        map route_find(\%connections, $_, $_, 3),
        keys %connections;
    @tri_loops = sort @tri_loops;

        scalar
        grep /\bt/,
        @tri_loops;
}

timer sub {
    say Dumper process_day23_part1($practice_input);
    # confirm process_day23_part1($practice_input), 7;
}

$VAR1 = 7;

[time] 0 seconds


0

Warning: Subroutine route_find redefined at reply input line 1.

Subroutine process_day23_part1 redefined at reply input line 21.


In [48]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/23/input');
    $res = process_day23_part1($input);
    say Dumper $res;
    confirm $res, 1151;
};

$VAR1 = 1151;

confirmed: 1151
[time] 3 seconds


3

In [47]:
say post_answer('2024/day/23/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 23 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [56]:
sub route_find {
    my ($map, $start, $end, $steps) = @_;
    my %connections = %$map;

    my @paths = { path => [ $start ], p => $start };
    foreach (1 .. $steps) {
        @paths = map {
            my $path = $_;
                map { { path => [ @{$path->{path}}, $_ ], p => $_ } }
                @{$connections{$path->{p}}}
        }
        @paths;
    }

    return
        map $_->{path},
        grep $_->{p} eq $end,
        @paths;
}

sub build_connection_table {
    my ($input) = @_;
    my @pairs = grep $_, split "\n", $input;
    my %connections;
    foreach (@pairs) {
        my ($from, $to) = split '-';
        push @{$connections{$from}}, $to;
        push @{$connections{$to}}, $from;
    }
    return \%connections;
}
sub build_nested_connection_table {
    my ($input) = @_;
    my @pairs = grep $_, split "\n", $input;
    my %connections;
    foreach (@pairs) {
        my ($from, $to) = split '-';
        $connections{$from}{$to} = 1;
        $connections{$to}{$from} = 1;
    }
    return \%connections;
}

sub find_tri_loops {
    my ($input) = @_;
    my $connection_table = build_connection_table($input);
    my @tri_loops =
        uniq
        map { join ',', sort @{$_}[1 .. 3] }
        map route_find($connection_table, $_, $_, 3),
        keys %$connection_table;
    @tri_loops = sort @tri_loops;

    @tri_loops;
}

Warning: Subroutine route_find redefined at reply input line 1.

Subroutine build_connection_table redefined at reply input line 21.

Subroutine find_tri_loops redefined at reply input line 44.


In [121]:
sub find_largest_network_for_key {
    my ($connection_table) = @_;
    my %con = %$connection_table;

    my @groups =
        map { my $key = $_; map [ $key, $_ ], keys %{$con{$key}} }
        keys %con;
    my $size = 2;
    while (@groups > 0) {
        appendfile('.exp/day23_networks', "\ncheckpoint $size:\n" . scalar(@groups) . "\n");
        $size++;
        my @next =
            # uniqueness
            map [ split ',' ], uniq map { join ',', sort @$_ }
            # group expansion
            map {
                my $group = $_;
                map { [ @$group, $_ ] }
                grep { my $key = $_; all sub { exists $con{$key}{$_} }, @$group }
                keys %{$con{$group->[-1]}}
            } @groups;
        return map { join ',', @$_ } @groups if @next == 0;
        @groups = @next;
    }

    die "should never occur";
}
sub process_day23_part2 {
    my ($input) = @_;
    my %con = %{build_nested_connection_table($input)};
    my ($largest) = find_largest_network_for_key(\%con);
    return $largest;
}

timer sub {
    say Dumper process_day23_part2($practice_input);
    # my $input = get_challenge('2024/day/23/input');
    # say Dumper process_day23_part2($input);
    # confirm process_day23_part2($practice_input), 37327623;
}

$VAR1 = 'co,de,ka,ta';

[time] 0 seconds


0

Warning: Subroutine find_largest_network_for_key redefined at reply input line 1.

Subroutine process_day23_part2 redefined at reply input line 28.


In [122]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/23/input');
    $res = process_day23_part2($input);
    say Dumper $res;
    # confirm $res, 19847565303;
};

$VAR1 = 'ar,cd,hl,iw,jm,ku,qo,rz,vo,xe,xm,xv,ys';

[time] 19 seconds


19

In [124]:
say post_answer('2024/day/23/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 23 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1